In [119]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import find_peaks
import csv

In [120]:
#read file
data = np.genfromtxt ('1216_ecg_data.csv', delimiter=',', dtype=np.float64)

In [126]:
filename = "data_preprocessing.csv"
with open(filename, 'w') as csvfile:
    csvwriter = csv.writer(csvfile) 
    for i in range(data.shape[0]):
        current_data = data[i, :data.shape[1] - 1]
        
        label = int(data[i, -1])
        peaks, _ = find_peaks(current_data, height=1.8)

        #find peak by average mid 1/3 difference of the sorted duration
        duration = []
        for j in range(len(peaks) - 1):
            duration.append(peaks[j + 1] - peaks[j])

        sort_duration = np.sort(duration)
        #print(sort_duration)
        period = int(np.mean(sort_duration[len(peaks)//3 : 2 * len(peaks) // 3]))

        #find the offset of the signal
        left = 0
        right = 0
        for j in range(len(peaks) - 1):
            if np.abs((peaks[j + 1] - peaks[j]) - period) < 2:
                left = peaks[j] 
                right = peaks[j + 1]
                break


        midpoint = (left + right) // 2
        offset =  midpoint  - (midpoint // period) * period

        ecg = []


        period_2_perform = int((len(current_data) - offset) // period)

        for j in range(period_2_perform - 1):
            temp = []
            for k in current_data[j * period + offset : (j + 1) * period + offset]:
                temp.append(k)
            ecg.append(temp)

        for j in range(len(ecg)):
            for k in range(186 - period):
                ecg[j] = np.append(ecg[j], [0])
            ecg[j] = np.append(ecg[j], [label])
            csvwriter.writerow(ecg[j])